In [1]:
!aws s3 rm s3://itversitydata/ghactivity --recursive

delete: s3://itversitydata/ghactivity/2022-06-01-0.json.gz


In [3]:
import boto3

In [4]:

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('jobs')
table.delete_item(Key={'job_id': 'ghactivity_ingest'})
item = {
    'job_id': 'ghactivity_ingest',
    'job_description': 'Ingest data from gmail to s3',
    'is_active': 'Y',
    'baseline_days': 3
}
table.put_item(Item=item)

{'ResponseMetadata': {'RequestId': '1FJ1VOQO15IOTUVS9TKPDMUUI3VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sat, 04 Jun 2022 01:16:37 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1FJ1VOQO15IOTUVS9TKPDMUUI3VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [5]:
def get_job_details(job_name):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('jobs')
    job_details = table.get_item(Key={'job_id': job_name})['Item']
    return job_details

In [6]:
get_job_details('ghactivity_ingest')    

{'job_description': 'Ingest data from gmail to s3',
 'is_active': 'Y',
 'job_id': 'ghactivity_ingest',
 'baseline_days': Decimal('3')}

In [7]:
from datetime import datetime as dt
from datetime import timedelta as td
import time

In [ ]:

def get_next_file_name(job_details):
    job_start_time = int(time.mktime(dt.now().timetuple()))
    job_run_bookmark_details = job_details.get('job_run_bookmark_details')
    if job_run_bookmark_details:
        dt_part = job_run_bookmark_details['last_run_file_name'].split('.')[0]
        next_file_name = f"{dt.strftime(dt.strptime(dt_part, '%Y-%m-%d-%H') + td(hours=1), '%Y-%m-%d-%-H')}.json.gz"
    else:
        next_file_name = f'{dt.strftime(dt.now().date() - td(days=3), "%Y-%m-%d")}-0.json.gz'
    return job_start_time, next_file_name

In [8]:
job_details = get_job_details('ghactivity_ingest')

In [11]:
job_start_time, next_file = get_next_file_name(job_details)

In [13]:
import requests

In [ ]:

def upload_file_to_s3(file_name):
   res = requests.get(f'https://data.gharchive.org/{file_name}')

   s3_client = boto3.client('s3')
   upload_res = s3_client.put_object(
      Bucket='itversitydata',
      Key=f'ghactivity/{file_name}',
      Body=res.content
   )

   return {
      'last_run_file_name': f's3://itversitydata/ghactivity/{file_name}',
      'status_code': upload_res['ResponseMetadata']['HTTPStatusCode']
   }


In [15]:
job_run_details = upload_file_to_s3(next_file)

In [18]:
def save_job_run_details(job_details, job_run_details, job_start_time):
    dynamodb = boto3.resource('dynamodb')
    job_run_details_item = {
        'job_id': job_details['job_id'],
        'job_run_time': job_start_time,
        'job_run_bookmark_details': job_run_details,
        'create_ts': int(time.mktime(dt.now().timetuple()))
    }
    job_run_details_table = dynamodb.Table('job_run_details')
    job_run_details_table.put_item(Item=job_run_details_item)
    
    job_details_table = dynamodb.Table('jobs')
    job_details['job_run_bookmark_details'] = job_run_details
    job_details_table.put_item(Item=job_details)

In [19]:
save_job_run_details(job_details, job_run_details, job_start_time)

In [15]:
!aws s3 ls s3://itversitydata/ghactivity/

2022-06-04 06:04:08   98882105 2022-06-01-0.json.gz
